# 지도 표시와 html 제작

In [1]:
import requests
import numpy as np
import pandas as pd
import folium
from folium.plugins import MiniMap

In [105]:
total = pd.read_excel('data_p/Tourist_spotTop10.xlsx')
total.columns = ['상호명','키워드수']
total.head()

,상호명,키워드수
0,매미성,56
1,정글돔,40
2,바람의언덕,37
3,샛바람소리길,11
4,해금강,11


In [106]:
# 거제 붙어있는 이름에서 '거제' 제거
total['상호명'] = total['상호명'].str.replace('거제','')
total

,상호명,키워드수
0,매미성,56
1,정글돔,40
2,바람의언덕,37
3,샛바람소리길,11
4,해금강,11
5,근포동굴,10
6,구조라성,9
7,조선해양문화관,6
8,지세포성,6
9,유호전망대,6


In [107]:
# 거제지역을 구하기 위해 앞에 거제를 붙힘
total['상호명'] = '거제 '+total['상호명']
total['상호명']

0        거제 매미성
1        거제 정글돔
2      거제 바람의언덕
3     거제 샛바람소리길
4        거제 해금강
5       거제 근포동굴
6       거제 구조라성
7    거제 조선해양문화관
8       거제 지세포성
9      거제 유호전망대
Name: 상호명, dtype: object

In [108]:
# 위치 정보의 종류를 확인하기 위해 list 형식으로 확인
locations = list(total['상호명'])
locations

['거제 매미성',
 '거제 정글돔',
 '거제 바람의언덕',
 '거제 샛바람소리길',
 '거제 해금강',
 '거제 근포동굴',
 '거제 구조라성',
 '거제 조선해양문화관',
 '거제 지세포성',
 '거제 유호전망대']

In [109]:
# 카카오맵 API를 이용한 장소 검색 함수 만들기

import requests
def find_places(searching):
    url = 'https://dapi.kakao.com/v2/local/search/keyword.json?query={}'.format(searching)
    headers = {'Authorization': 'KakaoAK 0706bf36cd54aff87ce0e2b03ce8370b'
    }
    
    result=[]
    foods = requests.get(url, headers = headers).json()['documents']
    for i in range(len(foods)):
        food = foods[i]
        category_name = food['category_name']
        p_url = food['place_url']
        name = food['place_name']
        x = food['x']
        y = food['y']
        address = food['road_address_name']
        data = [searching, category_name, p_url,name,x,y,address]
        
        result.append(data)

    return result

In [110]:
# tqdm을 이용해 작업하는 시간 알 수 있음.
from tqdm import tqdm
import time

inform = []
for location in tqdm(locations):
    try:
        data=find_places(location)
        inform += data
        time.sleep(0.5)
    except:
        pass
inform

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:05<00:00,  1.72it/s]


[['거제 매미성',
  '여행 > 관광,명소',
  'http://place.map.kakao.com/26459481',
  '매미성',
  '128.7049836287088',
  '34.96808253657398',
  ''],
 ['거제 매미성',
  '여행 > 숙박 > 펜션',
  'http://place.map.kakao.com/1277445329',
  '매미성하늘정원 펜션',
  '128.700908661215',
  '34.9654359409593',
  '경남 거제시 장목면 율천대금길 88-50'],
 ['거제 매미성',
  '음식점 > 한식 > 국수',
  'http://place.map.kakao.com/1680168170',
  '매미면가 매미성본점',
  '128.70351428423487',
  '34.9670216614856',
  '경남 거제시 장목면 복항길 2'],
 ['거제 매미성',
  '음식점 > 간식',
  'http://place.map.kakao.com/343690653',
  '미스터옥슈슈 매미성 본점',
  '128.70416049149657',
  '34.96703422287586',
  '경남 거제시 장목면 복항길 12'],
 ['거제 매미성',
  '금융,보험 > 금융서비스 > 은행 > ATM',
  'http://place.map.kakao.com/2107617367',
  '롯데ATM 세븐일레븐 거제매미성(CD)',
  '128.702309661143',
  '34.9681705283331',
  ''],
 ['거제 정글돔',
  '여행 > 관광,명소 > 수목원,식물원',
  'http://place.map.kakao.com/1413071040',
  '거제식물원 정글돔',
  '128.57822917272344',
  '34.85673294234183',
  '경남 거제시 거제면 거제남서로 3595'],
 ['거제 정글돔',
  '교통,수송 > 교통시설 > 주차장',
  'http://place.map.

In [112]:
# 데이터 프레임 변환 후 컬럼명 정해주고 엑셀 저장
inform_df = pd.DataFrame(inform)
inform_df.columns = ['검색어','카테고리','링크','상호명','경도','위도','주소']
inform_df.to_excel('data_p/여행지_kakaomap.xlsx', index = False)

In [113]:
inform_df = pd.read_excel('data_p/여행지_kakaomap.xlsx')

In [125]:
find_word = '관광'
cate = list(inform_df['카테고리'])
words1 = [i for i in range(len(cate)) if find_word in cate[i]]

words1

[0, 5, 8, 23, 24, 31, 39, 40, 44, 45]

In [126]:
find_word = '전시관'
cate = list(inform_df['카테고리'])
words2 = [i for i in range(len(cate)) if find_word in cate[i]]
print(words2)

[42, 43]


In [127]:
words = words1+words2
words.sort()

In [129]:
inform_df = inform_df.loc[words]
inform_df

,검색어,카테고리,링크,상호명,경도,위도,주소
0,거제 매미성,"여행 > 관광,명소",http://place.map.kakao.com/26459481,매미성,128.704984,34.968083,NaN
5,거제 정글돔,"여행 > 관광,명소 > 수목원,식물원",http://place.map.kakao.com/1413071040,거제식물원 정글돔,128.578229,34.856733,경남 거제시 거제면 거제남서로 3595
8,거제 바람의언덕,"여행 > 관광,명소 > 전망대",http://place.map.kakao.com/25035521,바람의언덕,128.663149,34.744621,NaN
23,거제 샛바람소리길,"여행 > 관광,명소 > 도보여행",http://place.map.kakao.com/110586060,샛바람소리길,128.693766,34.804704,NaN
24,거제 해금강,"여행 > 관광,명소 > 섬",http://place.map.kakao.com/10928180,해금강,128.683726,34.733299,NaN
31,거제 해금강,"여행 > 관광,명소 > 바위",http://place.map.kakao.com/848238167,해금강 사자바위,128.685416,34.736183,NaN
39,거제 근포동굴,"여행 > 관광,명소 > 동굴",http://place.map.kakao.com/878703964,근포동굴,128.590990,34.722944,NaN
40,거제 구조라성,"여행 > 관광,명소 > 문화유적 > 산성,성곽",http://place.map.kakao.com/8220494,구조라성,128.695052,34.802213,NaN
42,거제 조선해양문화관,"문화,예술 > 문화시설 > 전시관",http://place.map.kakao.com/12728251,조선해양문화관,128.701602,34.834870,경남 거제시 일운면 지세포해안로 25
43,거제 조선해양문화관,"문화,예술 > 문화시설 > 전시관",http://place.map.kakao.com/11919382,거제어촌민속전시관,128.701795,34.833605,경남 거제시 일운면 지세포해안로 41


In [130]:
inform_df = inform_df[inform_df['검색어'].duplicated()!=True]
inform_df = inform_df.reset_index(drop=True)
inform_df.to_excel('data_p/여행지_kakaomap.xlsx', index = False)
inform_df

,검색어,카테고리,링크,상호명,경도,위도,주소
0,거제 매미성,"여행 > 관광,명소",http://place.map.kakao.com/26459481,매미성,128.704984,34.968083,NaN
1,거제 정글돔,"여행 > 관광,명소 > 수목원,식물원",http://place.map.kakao.com/1413071040,거제식물원 정글돔,128.578229,34.856733,경남 거제시 거제면 거제남서로 3595
2,거제 바람의언덕,"여행 > 관광,명소 > 전망대",http://place.map.kakao.com/25035521,바람의언덕,128.663149,34.744621,NaN
3,거제 샛바람소리길,"여행 > 관광,명소 > 도보여행",http://place.map.kakao.com/110586060,샛바람소리길,128.693766,34.804704,NaN
4,거제 해금강,"여행 > 관광,명소 > 섬",http://place.map.kakao.com/10928180,해금강,128.683726,34.733299,NaN
5,거제 근포동굴,"여행 > 관광,명소 > 동굴",http://place.map.kakao.com/878703964,근포동굴,128.590990,34.722944,NaN
6,거제 구조라성,"여행 > 관광,명소 > 문화유적 > 산성,성곽",http://place.map.kakao.com/8220494,구조라성,128.695052,34.802213,NaN
7,거제 조선해양문화관,"문화,예술 > 문화시설 > 전시관",http://place.map.kakao.com/12728251,조선해양문화관,128.701602,34.834870,경남 거제시 일운면 지세포해안로 25
8,거제 지세포성,"여행 > 관광,명소 > 문화유적 > 산성,성곽",http://place.map.kakao.com/11275527,지세포성,128.718566,34.832958,NaN
9,거제 유호전망대,"여행 > 관광,명소 > 전망대",http://place.map.kakao.com/25051541,유호전망대,128.717495,35.015027,NaN


In [133]:
# 관광지의 경우 주소가 나오지 않는 현상이 생긴다 --> 카카오링크 들어가서 복붙하기
# 엑셀에서 바로 '여행지_kakaomap.xlsx'에 인스타그램 크롤링으로 뽑은 url, 순위 컬럼 추가하여 '여행지탑10.xlsx' 파일 만들기
pd.read_excel('data_p/여행지_kakaomap.xlsx')

,검색어,카테고리,링크,상호명,경도,위도,주소,URL,순위
0,거제 매미성,"여행 > 관광,명소",http://place.map.kakao.com/26459481,매미성,128.704984,34.968083,경남 거제시 장목면 대금리 290,https://www.instagram.com/p/CSMWmlhroTa/,1
1,거제 정글돔,"여행 > 관광,명소 > 수목원,식물원",http://place.map.kakao.com/1413071040,거제식물원 정글돔,128.578229,34.856733,경남 거제시 거제면 거제남서로 3595,https://www.instagram.com/p/B6uoov6l6qZ/,2
2,거제 바람의언덕,"여행 > 관광,명소 > 전망대",http://place.map.kakao.com/25035521,바람의언덕,128.663149,34.744621,경남 거제시 남부면 갈곶리 산 14-47 도장포마을,https://www.instagram.com/p/CVb9JCevXWo/,3
3,거제 샛바람소리길,"여행 > 관광,명소 > 도보여행",http://place.map.kakao.com/110586060,샛바람소리길,128.693766,34.804704,경남 거제시 일운면 구조라리 423-1,https://www.instagram.com/p/CVdAeQgPquf/,4
4,거제 해금강,"여행 > 관광,명소 > 섬",http://place.map.kakao.com/10928180,해금강,128.683726,34.733299,경남 거제시 남부면 갈곶리,https://www.instagram.com/p/CQAK70csF5h/,5
5,거제 근포동굴,"여행 > 관광,명소 > 동굴",http://place.map.kakao.com/878703964,근포동굴,128.590990,34.722944,경남 거제시 남부면 저구리 450-1,https://www.instagram.com/p/CSylh5dM4_e/,6
6,거제 구조라성,"여행 > 관광,명소 > 문화유적 > 산성,성곽",http://place.map.kakao.com/8220494,구조라성,128.695052,34.802213,경남 거제시 일운면 구조라리 산 55,https://www.instagram.com/p/CAesT5RH5xw/,7
7,거제 조선해양문화관,"문화,예술 > 문화시설 > 전시관",http://place.map.kakao.com/12728251,조선해양문화관,128.701602,34.834870,경남 거제시 일운면 지세포해안로 25,https://www.instagram.com/p/CT7Bg6HvaFK/,8
8,거제 지세포성,"여행 > 관광,명소 > 문화유적 > 산성,성곽",http://place.map.kakao.com/11275527,지세포성,128.718566,34.832958,경남 거제시 일운면 지세포리 235-4,https://www.instagram.com/p/CPo8eTlpm2j/,9
9,거제 유호전망대,"여행 > 관광,명소 > 전망대",http://place.map.kakao.com/25051541,유호전망대,128.717495,35.015027,경남 거제시 장목면 유호리 산 85-2,https://www.instagram.com/p/CT1T_5-Jwhj/,10


In [138]:
inform_df = pd.read_excel('data_p/여행지_kakaomap.xlsx')
inform_df

,검색어,카테고리,링크,상호명,경도,위도,주소,URL,순위
0,거제 매미성,"여행 > 관광,명소",http://place.map.kakao.com/26459481,매미성,128.704984,34.968083,경남 거제시 장목면 대금리 290,https://www.instagram.com/p/CSMWmlhroTa/,1
1,거제 정글돔,"여행 > 관광,명소 > 수목원,식물원",http://place.map.kakao.com/1413071040,거제식물원 정글돔,128.578229,34.856733,경남 거제시 거제면 거제남서로 3595,https://www.instagram.com/p/B6uoov6l6qZ/,2
2,거제 바람의언덕,"여행 > 관광,명소 > 전망대",http://place.map.kakao.com/25035521,바람의언덕,128.663149,34.744621,경남 거제시 남부면 갈곶리 산 14-47 도장포마을,https://www.instagram.com/p/CVb9JCevXWo/,3
3,거제 샛바람소리길,"여행 > 관광,명소 > 도보여행",http://place.map.kakao.com/110586060,샛바람소리길,128.693766,34.804704,경남 거제시 일운면 구조라리 423-1,https://www.instagram.com/p/CVdAeQgPquf/,4
4,거제 해금강,"여행 > 관광,명소 > 섬",http://place.map.kakao.com/10928180,해금강,128.683726,34.733299,경남 거제시 남부면 갈곶리,https://www.instagram.com/p/CQAK70csF5h/,5
5,거제 근포동굴,"여행 > 관광,명소 > 동굴",http://place.map.kakao.com/878703964,근포동굴,128.590990,34.722944,경남 거제시 남부면 저구리 450-1,https://www.instagram.com/p/CSylh5dM4_e/,6
6,거제 구조라성,"여행 > 관광,명소 > 문화유적 > 산성,성곽",http://place.map.kakao.com/8220494,구조라성,128.695052,34.802213,경남 거제시 일운면 구조라리 산 55,https://www.instagram.com/p/CAesT5RH5xw/,7
7,거제 조선해양문화관,"문화,예술 > 문화시설 > 전시관",http://place.map.kakao.com/12728251,조선해양문화관,128.701602,34.834870,경남 거제시 일운면 지세포해안로 25,https://www.instagram.com/p/CT7Bg6HvaFK/,8
8,거제 지세포성,"여행 > 관광,명소 > 문화유적 > 산성,성곽",http://place.map.kakao.com/11275527,지세포성,128.718566,34.832958,경남 거제시 일운면 지세포리 235-4,https://www.instagram.com/p/CPo8eTlpm2j/,9
9,거제 유호전망대,"여행 > 관광,명소 > 전망대",http://place.map.kakao.com/25051541,유호전망대,128.717495,35.015027,경남 거제시 장목면 유호리 산 85-2,https://www.instagram.com/p/CT1T_5-Jwhj/,10


In [139]:
# 필요한 데이터만 리스트 안에 딕셔너리 형태로 만들기
info_list=[]
for i in range(len(inform_df)):
    info={}
    lat = inform_df.loc[i,'위도']
    long = inform_df.loc[i,'경도']
    insta_url = inform_df.loc[i,'URL']
    kakao_url = inform_df.loc[i,'링크']
    rank = inform_df.loc[i,'순위']
    
    info[inform_df.loc[i,'상호명']] = [(lat, long),  insta_url, kakao_url, rank]
    info_list.append(info)
    
info_list

[{'매미성': [(34.96808253657398, 128.7049836287088),
   'https://www.instagram.com/p/CSMWmlhroTa/',
   'http://place.map.kakao.com/26459481',
   1]},
 {'거제식물원 정글돔': [(34.85673294234183, 128.5782291727234),
   'https://www.instagram.com/p/B6uoov6l6qZ/',
   'http://place.map.kakao.com/1413071040',
   2]},
 {'바람의언덕': [(34.7446210175676, 128.663149179091),
   'https://www.instagram.com/p/CVb9JCevXWo/',
   'http://place.map.kakao.com/25035521',
   3]},
 {'샛바람소리길': [(34.8047035701488, 128.693765972468),
   'https://www.instagram.com/p/CVdAeQgPquf/',
   'http://place.map.kakao.com/110586060',
   4]},
 {'해금강': [(34.7332992716928, 128.683726401279),
   'https://www.instagram.com/p/CQAK70csF5h/',
   'http://place.map.kakao.com/10928180',
   5]},
 {'근포동굴': [(34.72294372603651, 128.590989560778),
   'https://www.instagram.com/p/CSylh5dM4_e/',
   'http://place.map.kakao.com/878703964',
   6]},
 {'구조라성': [(34.80221255800077, 128.6950523957659),
   'https://www.instagram.com/p/CAesT5RH5xw/',
   'http://

In [140]:
# 지도 만들기
from folium.plugins import MarkerCluster

m = folium.Map(location=[34.863537260180124, 128.62939119809138], zoom_start=11)

for data in info_list : 
    for place, details in data.items() :
        name = place
        coordinates = details[0]
        insta_post = details[1]
        website = details[2]
        rank = details[3]

        # Define html inside marker pop-up
        place_html = folium.Html(f"""<style>@import url('https://fonts.googleapis.com/css2?family=Gugi&display=swap');</style><p style="text-align: center;"><span style="font-family: 'Gugi', cursive; font-size: 22px;">{rank}위: {name}</span></p>
        <p style="text-align: center;"><iframe src={insta_post}embed width="240" height="290" frameborder="0" scrolling="auto" allowtransparency="true"></iframe>
        <p style="text-align: center;"><a href={website} target="_blank" title="{name} Website"><span style="font-family: 'Gugi', cursive; font-size: 13px;">{name} >>카카오맵 상세정보 보기</span></a></p>   
        """, script=True)
        # Create pop-up with html content
        popup = folium.Popup(place_html, max_width=700)
        
        folium.Marker(
            location=coordinates, 
            tooltip=name, 
            popup=popup,
            icon=folium.Icon(color='blue',icon='star')
        ).add_to(m)
        

    
m.save('data_p/Tourist_spot_Top10.html')

## 카테고리(맛집, 카페, 여행지, 숙박)별로 같은 작업 수행하여 지도 생성
**예시로 카페만 올렸어요(data_p 폴더에 완성본 html파일 있습니다)**